In [183]:
%matplotlib inline

In [184]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn import metrics

In [388]:
train_df = pd.read_csv('../data/train/train_feature_21to33_14_17_19_20_norm_2.csv', header = 0, encoding = 'utf-8')
test_a_df = pd.read_csv('../data/Test-A/test_a_feature_21to33_14_17_19_20_norm_2.csv', header = 0, encoding = 'utf-8')

In [389]:
trains = train_df

In [390]:
trains.drop_duplicates(inplace=True)

In [391]:
online_test = test_a_df

In [392]:
from sklearn.model_selection import train_test_split
train_xy,offline_test = train_test_split(trains,test_size = 0.2,random_state=21)
train,val = train_test_split(train_xy,test_size = 0.2,random_state=21)

In [393]:
y_train = train.label
X_train = train.drop(['uid','label'],axis=1)

In [394]:
y_val = val.label
X_val = val.drop(['uid','label'],axis=1)

In [395]:
offline_test_X = offline_test.drop(['uid','label'],axis=1)
online_test_X  = online_test.drop(['uid'],axis=1)

# 数据转换

In [396]:
lgb_train = lgb.Dataset(X_train, y_train, free_raw_data=False)
lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train,free_raw_data=False)

# 设置参数

In [397]:
params = {
          'boosting_type': 'gbdt',
          'objective': 'binary',
          'metric': 'auc',
        'metric_freq': 100,
        'is_training_metric': True,
        'min_data_in_leaf': 360,
        'num_leaves': 50,
        'learning_rate': 0.08,
        'is_unbalance': True,
        'feature_fraction': 0.8,
        'bagging_fraction': 0.8,
        'min_hessian': 0.05,
         'verbosity':-1
}

# 交叉验证

In [143]:
def evalMetric(preds,dtrain):
    
    label = dtrain.get_label()
    
    
    pre = pd.DataFrame({'preds':preds,'label':label})
    pre= pre.sort_values(by='preds',ascending=False)
    
    auc = metrics.roc_auc_score(pre.label,pre.preds)

    pre.preds=pre.preds.map(lambda x: 1 if x>=0.5 else 0)

    f1 = metrics.f1_score(pre.label,pre.preds)
    
    
    res = 0.6*auc +0.4*f1
    
    return 'res',res,True

In [144]:
print(X_train)

                 1         2         3   4             5             6  \
237   7.011348e-02  0.037568  0.008529   0  6.762647e-02  1.102896e-01   
806   3.533295e-02  0.067908  0.018238   0  8.492208e-02  8.930566e-03   
133   2.767749e-02  0.027132  0.011741   0  3.580039e-02  8.945542e-02   
4520  2.544959e-02  0.007241  0.001510   0  2.480089e-02  4.741997e-03   
3978  4.572688e-02  0.009737  0.010100   0  2.563733e-02  2.795152e-02   
2687  1.232327e-01  0.043892  0.009042   0  8.820225e-02  1.515367e-01   
3235  5.219418e-03  0.005229  0.014167   0  6.499558e-03  4.021916e-03   
644   6.479168e-03  0.003161  0.002699   0  5.735066e-03  4.169911e-03   
3688  5.846238e-02  0.032366  0.009998   0  6.469776e-02  6.879154e-02   
1935  1.850997e-02  0.013731  0.006142   0  1.989921e-02  2.217944e-02   
3731  3.728731e-02  0.008087  0.008910   0  2.032930e-02  2.101268e-02   
2805  1.180626e-04  0.000530  0.004218   0  3.638367e-04  5.768281e-05   
321   8.232765e-08  0.000001  0.000000

# 调参1

In [150]:
max_mean = float(0)
best_params = {}

In [151]:
for num_leaves in range(50, 55, 5):
    for learning_rate in np.arange(0.05, 1.0, 0.05):
        params['num_leaves'] = num_leaves
        # params['max_depth'] = max_depth
        params['learning_rate'] = learning_rate
        
        cv_results = lgb.cv(
                            params,
                            lgb_train,
                            seed=2018,
                            nfold=3,
                            feval=evalMetric,
                            metrics=['evalMetric'],
                            early_stopping_rounds=100,
                            verbose_eval=5,
                            num_boost_round=10000,
                            )

        res_mean = pd.Series(cv_results['res-mean']).max()
        # boost_rounds = pd.Series(cv_results['binary_error-mean']).argmin()
        print(res_mean)

        if res_mean > max_mean:
            print(num_leaves, learning_rate)
            max_mean = res_mean
            best_params['num_leaves'] = num_leaves
            # best_params['max_depth'] = max_depth
            best_params['learning_rate'] = learning_rate

params['num_leaves'] = best_params['num_leaves']
# params['max_depth'] = best_params['max_depth']
params['learning_rate'] = best_params['learning_rate']

[5]	cv_agg's res: 0.720535 + 0.00613134
[10]	cv_agg's res: 0.726229 + 0.00564088
[15]	cv_agg's res: 0.732958 + 0.00900984
[20]	cv_agg's res: 0.733598 + 0.00779738
[25]	cv_agg's res: 0.734477 + 0.00506384
[30]	cv_agg's res: 0.739878 + 0.00747321
[35]	cv_agg's res: 0.74776 + 0.0083774
[40]	cv_agg's res: 0.751327 + 0.00904721
[45]	cv_agg's res: 0.753728 + 0.0106669
[50]	cv_agg's res: 0.756402 + 0.0114539
[55]	cv_agg's res: 0.758314 + 0.0107705
[60]	cv_agg's res: 0.761588 + 0.0101643
[65]	cv_agg's res: 0.764068 + 0.0104747
[70]	cv_agg's res: 0.766832 + 0.0092796
[75]	cv_agg's res: 0.766756 + 0.00883821
[80]	cv_agg's res: 0.768626 + 0.00928067
[85]	cv_agg's res: 0.771907 + 0.00910013
[90]	cv_agg's res: 0.773397 + 0.0105264
[95]	cv_agg's res: 0.77594 + 0.0118361
[100]	cv_agg's res: 0.776362 + 0.012617
[105]	cv_agg's res: 0.778933 + 0.0118457
[110]	cv_agg's res: 0.780287 + 0.0127258
[115]	cv_agg's res: 0.780574 + 0.0133924
[120]	cv_agg's res: 0.781953 + 0.011863
[125]	cv_agg's res: 0.784236 +

[65]	cv_agg's res: 0.787145 + 0.0120592
[70]	cv_agg's res: 0.787959 + 0.0113293
[75]	cv_agg's res: 0.787261 + 0.0122163
[80]	cv_agg's res: 0.787528 + 0.0107464
[85]	cv_agg's res: 0.787144 + 0.0110716
[90]	cv_agg's res: 0.788831 + 0.0130858
[95]	cv_agg's res: 0.787978 + 0.0138931
[100]	cv_agg's res: 0.787123 + 0.0139602
[105]	cv_agg's res: 0.789745 + 0.0134245
[110]	cv_agg's res: 0.787665 + 0.0135837
[115]	cv_agg's res: 0.788257 + 0.0133858
[120]	cv_agg's res: 0.788983 + 0.0126012
[125]	cv_agg's res: 0.789079 + 0.0126916
[130]	cv_agg's res: 0.789654 + 0.0137277
[135]	cv_agg's res: 0.787038 + 0.0141247
[140]	cv_agg's res: 0.787391 + 0.0153719
[145]	cv_agg's res: 0.788567 + 0.014907
[150]	cv_agg's res: 0.788777 + 0.0140529
[155]	cv_agg's res: 0.78854 + 0.0154744
[160]	cv_agg's res: 0.786807 + 0.0161493
[165]	cv_agg's res: 0.786271 + 0.0169479
[170]	cv_agg's res: 0.785608 + 0.0166588
[175]	cv_agg's res: 0.786766 + 0.0156801
[180]	cv_agg's res: 0.786829 + 0.0149219
[185]	cv_agg's res: 0.784

[105]	cv_agg's res: 0.782338 + 0.013243
[110]	cv_agg's res: 0.780189 + 0.0120175
[115]	cv_agg's res: 0.779959 + 0.0121328
[120]	cv_agg's res: 0.781434 + 0.0147123
[125]	cv_agg's res: 0.779467 + 0.0143184
[130]	cv_agg's res: 0.778313 + 0.0142125
0.7878954962517795
[5]	cv_agg's res: 0.752067 + 0.0129415
[10]	cv_agg's res: 0.769789 + 0.0110628
[15]	cv_agg's res: 0.778177 + 0.013492
[20]	cv_agg's res: 0.777237 + 0.0155626
[25]	cv_agg's res: 0.781006 + 0.0117414
[30]	cv_agg's res: 0.781117 + 0.0110105
[35]	cv_agg's res: 0.781285 + 0.0139311
[40]	cv_agg's res: 0.784143 + 0.0141641
[45]	cv_agg's res: 0.778589 + 0.0128616
[50]	cv_agg's res: 0.778228 + 0.0156556
[55]	cv_agg's res: 0.778707 + 0.0106467
[60]	cv_agg's res: 0.777862 + 0.0120416
[65]	cv_agg's res: 0.777647 + 0.0111407
[70]	cv_agg's res: 0.782698 + 0.0156333
[75]	cv_agg's res: 0.780605 + 0.0175281
[80]	cv_agg's res: 0.780974 + 0.0160471
[85]	cv_agg's res: 0.778833 + 0.01504
[90]	cv_agg's res: 0.777843 + 0.0168245
[95]	cv_agg's res: 0

[55]	cv_agg's res: 0.759893 + 0.0202566
[60]	cv_agg's res: 0.759914 + 0.0194713
[65]	cv_agg's res: 0.758283 + 0.0196342
[70]	cv_agg's res: 0.755769 + 0.0214
[75]	cv_agg's res: 0.760956 + 0.0184169
[80]	cv_agg's res: 0.756936 + 0.0185709
[85]	cv_agg's res: 0.755412 + 0.0173602
[90]	cv_agg's res: 0.754362 + 0.0205427
[95]	cv_agg's res: 0.752571 + 0.0203686
[100]	cv_agg's res: 0.754033 + 0.0226757
[105]	cv_agg's res: 0.75288 + 0.0181198
[110]	cv_agg's res: 0.753316 + 0.0207635
[115]	cv_agg's res: 0.752424 + 0.0214664
0.771753424251098
[5]	cv_agg's res: 0.743623 + 0.0194862
[10]	cv_agg's res: 0.763238 + 0.00593259
[15]	cv_agg's res: 0.766003 + 0.00769709
[20]	cv_agg's res: 0.763784 + 0.016454
[25]	cv_agg's res: 0.768048 + 0.0154699
[30]	cv_agg's res: 0.764667 + 0.0144229
[35]	cv_agg's res: 0.765025 + 0.0150419
[40]	cv_agg's res: 0.765743 + 0.0175156
[45]	cv_agg's res: 0.762596 + 0.0134271
[50]	cv_agg's res: 0.762554 + 0.0174392
[55]	cv_agg's res: 0.761916 + 0.0187458
[60]	cv_agg's res: 0.7

In [152]:
print(params['num_leaves'], params['learning_rate'])

(50, 0.15000000000000002)


# 调参2

In [155]:
for max_bin in range(255,256,1):
    for min_data_in_leaf in range(300,500,10):
            params['max_bin'] = max_bin
            params['min_data_in_leaf'] = min_data_in_leaf
            
            cv_results = lgb.cv(
                            params,
                            lgb_train,
                            seed=42,
                            nfold=3,
                            feval=evalMetric,
                            metrics=['evalMetric'],
                            early_stopping_rounds=100,
                            verbose_eval=5,
                            num_boost_round=10000,
                            )
                    
            res_mean = pd.Series(cv_results['res-mean']).max()
            # boost_rounds = pd.Series(cv_results['binary_error-mean']).argmin()
            print(res_mean)

            if res_mean > max_mean:
                print(max_bin, min_data_in_leaf)
                max_mean = res_mean
                best_params['max_bin']= max_bin
                best_params['min_data_in_leaf'] = min_data_in_leaf

params['min_data_in_leaf'] = best_params['min_data_in_leaf']
params['max_bin'] = best_params['max_bin']

[5]	cv_agg's res: 0.726901 + 0.00908528
[10]	cv_agg's res: 0.748482 + 0.00189022
[15]	cv_agg's res: 0.759139 + 0.000926309
[20]	cv_agg's res: 0.773174 + 0.000803587
[25]	cv_agg's res: 0.77943 + 0.0029423
[30]	cv_agg's res: 0.784452 + 0.00310359
[35]	cv_agg's res: 0.786384 + 0.00139413
[40]	cv_agg's res: 0.787799 + 0.00126923
[45]	cv_agg's res: 0.788947 + 0.00251496
[50]	cv_agg's res: 0.787275 + 0.00339593
[55]	cv_agg's res: 0.78723 + 0.00354147
[60]	cv_agg's res: 0.789643 + 0.00257465
[65]	cv_agg's res: 0.789362 + 0.00163762
[70]	cv_agg's res: 0.790808 + 0.00298062
[75]	cv_agg's res: 0.791429 + 0.00244044
[80]	cv_agg's res: 0.791246 + 0.0032344
[85]	cv_agg's res: 0.788884 + 0.00243493
[90]	cv_agg's res: 0.788848 + 0.00072878
[95]	cv_agg's res: 0.788484 + 0.00137242
[100]	cv_agg's res: 0.789822 + 0.0026539
[105]	cv_agg's res: 0.791937 + 0.00232567
[110]	cv_agg's res: 0.792705 + 0.00252079
[115]	cv_agg's res: 0.792858 + 0.00258009
[120]	cv_agg's res: 0.789983 + 0.00292284
[125]	cv_agg's 

[60]	cv_agg's res: 0.789691 + 0.00275256
[65]	cv_agg's res: 0.789199 + 0.00210834
[70]	cv_agg's res: 0.787403 + 0.00126704
[75]	cv_agg's res: 0.788268 + 0.00259056
[80]	cv_agg's res: 0.788624 + 0.00263393
[85]	cv_agg's res: 0.789866 + 0.00297197
[90]	cv_agg's res: 0.789156 + 0.00201689
[95]	cv_agg's res: 0.788356 + 0.00350549
[100]	cv_agg's res: 0.786912 + 0.00280674
[105]	cv_agg's res: 0.788272 + 0.00200897
[110]	cv_agg's res: 0.788336 + 0.00317444
[115]	cv_agg's res: 0.787991 + 0.00196997
[120]	cv_agg's res: 0.787656 + 0.0019523
[125]	cv_agg's res: 0.78774 + 0.00015395
[130]	cv_agg's res: 0.788036 + 0.000539051
[135]	cv_agg's res: 0.788915 + 0.00225135
[140]	cv_agg's res: 0.786791 + 2.78742e-05
[145]	cv_agg's res: 0.788149 + 0.00196316
[150]	cv_agg's res: 0.787739 + 0.0014521
[155]	cv_agg's res: 0.786687 + 0.00112642
[160]	cv_agg's res: 0.787112 + 0.000720825
0.7904709033991462
[5]	cv_agg's res: 0.718164 + 0.00428196
[10]	cv_agg's res: 0.740373 + 0.00406899
[15]	cv_agg's res: 0.75140

[240]	cv_agg's res: 0.786385 + 0.003122
[245]	cv_agg's res: 0.786293 + 0.00294295
[250]	cv_agg's res: 0.786779 + 0.00384623
[255]	cv_agg's res: 0.78569 + 0.00274462
[260]	cv_agg's res: 0.787001 + 0.00242935
[265]	cv_agg's res: 0.78574 + 0.00133162
[270]	cv_agg's res: 0.786236 + 0.00160524
[275]	cv_agg's res: 0.786602 + 0.0027379
[280]	cv_agg's res: 0.786894 + 0.0035873
[285]	cv_agg's res: 0.78579 + 0.0029141
[290]	cv_agg's res: 0.78449 + 0.0044369
[295]	cv_agg's res: 0.785658 + 0.00419124
[300]	cv_agg's res: 0.786372 + 0.00418879
[305]	cv_agg's res: 0.78499 + 0.00515902
[310]	cv_agg's res: 0.784929 + 0.00581353
0.788786736300935
[5]	cv_agg's res: 0.714827 + 0.00697735
[10]	cv_agg's res: 0.735473 + 0.00781766
[15]	cv_agg's res: 0.750866 + 0.00219647
[20]	cv_agg's res: 0.760785 + 0.00359192
[25]	cv_agg's res: 0.767187 + 0.00513391
[30]	cv_agg's res: 0.776996 + 0.00596551
[35]	cv_agg's res: 0.781231 + 0.0027179
[40]	cv_agg's res: 0.781236 + 0.00323009
[45]	cv_agg's res: 0.783761 + 0.00300

[70]	cv_agg's res: 0.783755 + 0.00628092
[75]	cv_agg's res: 0.784671 + 0.0033424
[80]	cv_agg's res: 0.78588 + 0.00253113
[85]	cv_agg's res: 0.785155 + 0.00277894
[90]	cv_agg's res: 0.785978 + 0.00420812
[95]	cv_agg's res: 0.785177 + 0.00284665
[100]	cv_agg's res: 0.784295 + 0.00183948
[105]	cv_agg's res: 0.784737 + 0.00288004
[110]	cv_agg's res: 0.785216 + 0.00139048
[115]	cv_agg's res: 0.785204 + 0.00278687
[120]	cv_agg's res: 0.783705 + 0.00192577
[125]	cv_agg's res: 0.784456 + 0.00255443
[130]	cv_agg's res: 0.785098 + 0.00314168
[135]	cv_agg's res: 0.784634 + 0.00363768
[140]	cv_agg's res: 0.782947 + 0.0042485
[145]	cv_agg's res: 0.782973 + 0.00340187
[150]	cv_agg's res: 0.783171 + 0.00419747
[155]	cv_agg's res: 0.782865 + 0.0046552
[160]	cv_agg's res: 0.782308 + 0.00481633
[165]	cv_agg's res: 0.781251 + 0.00358464
[170]	cv_agg's res: 0.782084 + 0.0038193
[175]	cv_agg's res: 0.783915 + 0.00378819
[180]	cv_agg's res: 0.782245 + 0.00269515
0.786123346041857
[5]	cv_agg's res: 0.721194 

[5]	cv_agg's res: 0.716901 + 0.0110077
[10]	cv_agg's res: 0.723747 + 0.014022
[15]	cv_agg's res: 0.739854 + 0.00374437
[20]	cv_agg's res: 0.744524 + 0.00237181
[25]	cv_agg's res: 0.752448 + 0.00239738
[30]	cv_agg's res: 0.755006 + 0.00437631
[35]	cv_agg's res: 0.75811 + 0.00387498
[40]	cv_agg's res: 0.761046 + 0.00639637
[45]	cv_agg's res: 0.766259 + 0.00670072
[50]	cv_agg's res: 0.764743 + 0.00660861
[55]	cv_agg's res: 0.76631 + 0.00499618
[60]	cv_agg's res: 0.767302 + 0.00616927
[65]	cv_agg's res: 0.768903 + 0.0071854
[70]	cv_agg's res: 0.769264 + 0.00515026
[75]	cv_agg's res: 0.769881 + 0.00453269
[80]	cv_agg's res: 0.769826 + 0.00615133
[85]	cv_agg's res: 0.770603 + 0.00500901
[90]	cv_agg's res: 0.771977 + 0.00336445
[95]	cv_agg's res: 0.772091 + 0.00367595
[100]	cv_agg's res: 0.773513 + 0.00385668
[105]	cv_agg's res: 0.773583 + 0.00440823
[110]	cv_agg's res: 0.773068 + 0.00370231
[115]	cv_agg's res: 0.774264 + 0.00313719
[120]	cv_agg's res: 0.775074 + 0.00291976
[125]	cv_agg's res

In [156]:
print(params['min_data_in_leaf'], params['max_bin'])

(360, 255)


# 调参3

In [157]:
for feature_fraction in [0.5,0.6,0.7,0.8,0.9,1.0]:
    for bagging_fraction in [0.5,0.6,0.7,0.8,0.9,1.0]:
        for bagging_freq in range(0,11,5):
            params['feature_fraction'] = feature_fraction
            params['bagging_fraction'] = bagging_fraction
            params['bagging_freq'] = bagging_freq
            
            cv_results = lgb.cv(
                            params,
                            lgb_train,
                            seed=42,
                            nfold=3,
                            feval=evalMetric,
                            metrics=['evalMetric'],
                            early_stopping_rounds=100,
                            verbose_eval=5,
                            num_boost_round=10000,
                            )
                    
            res_mean = pd.Series(cv_results['res-mean']).max()
            # boost_rounds = pd.Series(cv_results['binary_error-mean']).argmin()
            print(res_mean)

            if res_mean > max_mean:
                print(feature_fraction, bagging_fraction, bagging_freq)
                max_mean = res_mean
                best_params['feature_fraction'] = feature_fraction
                best_params['bagging_fraction'] = bagging_fraction
                best_params['bagging_freq'] = bagging_freq

params['feature_fraction'] = best_params['feature_fraction']
params['bagging_fraction'] = best_params['bagging_fraction']
params['bagging_freq'] = best_params['bagging_freq']

[5]	cv_agg's res: 0.707921 + 0.0075192
[10]	cv_agg's res: 0.738441 + 0.00122179
[15]	cv_agg's res: 0.759398 + 0.00592735
[20]	cv_agg's res: 0.765425 + 0.00488099
[25]	cv_agg's res: 0.772453 + 0.00166284
[30]	cv_agg's res: 0.778163 + 0.00341226
[35]	cv_agg's res: 0.778204 + 0.000327565
[40]	cv_agg's res: 0.780601 + 0.0024123
[45]	cv_agg's res: 0.784142 + 0.0036285
[50]	cv_agg's res: 0.785194 + 0.00294762
[55]	cv_agg's res: 0.785536 + 0.00150489
[60]	cv_agg's res: 0.785689 + 0.00100733
[65]	cv_agg's res: 0.785011 + 0.000373971
[70]	cv_agg's res: 0.785111 + 0.00134412
[75]	cv_agg's res: 0.78763 + 0.00235715
[80]	cv_agg's res: 0.787355 + 0.00179346
[85]	cv_agg's res: 0.785994 + 0.00223073
[90]	cv_agg's res: 0.787423 + 0.00118738
[95]	cv_agg's res: 0.787759 + 0.00224049
[100]	cv_agg's res: 0.785207 + 0.00372089
[105]	cv_agg's res: 0.783829 + 0.00420109
[110]	cv_agg's res: 0.785044 + 0.00340236
[115]	cv_agg's res: 0.786483 + 0.00287745
[120]	cv_agg's res: 0.785656 + 0.0031478
[125]	cv_agg's 

[310]	cv_agg's res: 0.761846 + 0.00776097
[315]	cv_agg's res: 0.761464 + 0.00709279
[320]	cv_agg's res: 0.760891 + 0.00750222
[325]	cv_agg's res: 0.760739 + 0.0058296
[330]	cv_agg's res: 0.759708 + 0.00797133
[335]	cv_agg's res: 0.759251 + 0.00874414
[340]	cv_agg's res: 0.759143 + 0.00966955
[345]	cv_agg's res: 0.758237 + 0.00751011
[350]	cv_agg's res: 0.759155 + 0.00549523
[355]	cv_agg's res: 0.757543 + 0.00673359
[360]	cv_agg's res: 0.756129 + 0.00688339
[365]	cv_agg's res: 0.758538 + 0.0071642
[370]	cv_agg's res: 0.758551 + 0.0073991
[375]	cv_agg's res: 0.760125 + 0.00730927
[380]	cv_agg's res: 0.760366 + 0.00593811
[385]	cv_agg's res: 0.760023 + 0.00833207
[390]	cv_agg's res: 0.760205 + 0.00835529
[395]	cv_agg's res: 0.761226 + 0.0033112
[400]	cv_agg's res: 0.7582 + 0.00384599
[405]	cv_agg's res: 0.758939 + 0.00311304
[410]	cv_agg's res: 0.758172 + 0.00485483
[415]	cv_agg's res: 0.759252 + 0.0055062
0.7623299400172233
[5]	cv_agg's res: 0.707921 + 0.0075192
[10]	cv_agg's res: 0.7384

[165]	cv_agg's res: 0.785284 + 0.00416693
[170]	cv_agg's res: 0.78563 + 0.00241882
[175]	cv_agg's res: 0.786519 + 0.00216686
[180]	cv_agg's res: 0.784656 + 0.00135367
[185]	cv_agg's res: 0.784628 + 0.00358991
[190]	cv_agg's res: 0.784256 + 0.00302529
0.7884655885683486
[5]	cv_agg's res: 0.671636 + 0.00892194
[10]	cv_agg's res: 0.707035 + 0.00885728
[15]	cv_agg's res: 0.729776 + 0.00136696
[20]	cv_agg's res: 0.73506 + 0.00186565
[25]	cv_agg's res: 0.74638 + 0.000982744
[30]	cv_agg's res: 0.751219 + 0.0021132
[35]	cv_agg's res: 0.756208 + 0.000902758
[40]	cv_agg's res: 0.759695 + 0.00504263
[45]	cv_agg's res: 0.763662 + 0.00486704
[50]	cv_agg's res: 0.763904 + 0.0071979
[55]	cv_agg's res: 0.7663 + 0.00802818
[60]	cv_agg's res: 0.766068 + 0.00679478
[65]	cv_agg's res: 0.769914 + 0.00520932
[70]	cv_agg's res: 0.769536 + 0.00603432
[75]	cv_agg's res: 0.770468 + 0.0078147
[80]	cv_agg's res: 0.768783 + 0.00643919
[85]	cv_agg's res: 0.769826 + 0.00702264
[90]	cv_agg's res: 0.770046 + 0.0098160

[5]	cv_agg's res: 0.702957 + 0.00944101
[10]	cv_agg's res: 0.727195 + 0.0125491
[15]	cv_agg's res: 0.743231 + 0.0102878
[20]	cv_agg's res: 0.749522 + 0.00902783
[25]	cv_agg's res: 0.757297 + 0.00654895
[30]	cv_agg's res: 0.761299 + 0.00295401
[35]	cv_agg's res: 0.768125 + 0.00326774
[40]	cv_agg's res: 0.771397 + 0.00567553
[45]	cv_agg's res: 0.772305 + 0.00677431
[50]	cv_agg's res: 0.774113 + 0.00443411
[55]	cv_agg's res: 0.775857 + 0.00463002
[60]	cv_agg's res: 0.775719 + 0.00498913
[65]	cv_agg's res: 0.776983 + 0.00297978
[70]	cv_agg's res: 0.777426 + 0.00421842
[75]	cv_agg's res: 0.776198 + 0.00205054
[80]	cv_agg's res: 0.777292 + 0.00434052
[85]	cv_agg's res: 0.778903 + 0.00304022
[90]	cv_agg's res: 0.78037 + 0.0022102
[95]	cv_agg's res: 0.780273 + 0.000809976
[100]	cv_agg's res: 0.783333 + 0.00210482
[105]	cv_agg's res: 0.782335 + 0.00317363
[110]	cv_agg's res: 0.780878 + 0.0030789
[115]	cv_agg's res: 0.780552 + 0.00351429
[120]	cv_agg's res: 0.780505 + 0.00378603
[125]	cv_agg's r

[45]	cv_agg's res: 0.782805 + 0.00737288
[50]	cv_agg's res: 0.783896 + 0.00713254
[55]	cv_agg's res: 0.787069 + 0.00517506
[60]	cv_agg's res: 0.786258 + 0.00486263
[65]	cv_agg's res: 0.785592 + 0.00719515
[70]	cv_agg's res: 0.787399 + 0.0078496
[75]	cv_agg's res: 0.785839 + 0.00719846
[80]	cv_agg's res: 0.785317 + 0.00763894
[85]	cv_agg's res: 0.785362 + 0.00599412
[90]	cv_agg's res: 0.786715 + 0.00639067
[95]	cv_agg's res: 0.786348 + 0.00625179
[100]	cv_agg's res: 0.790213 + 0.00603571
[105]	cv_agg's res: 0.789817 + 0.00559006
[110]	cv_agg's res: 0.791747 + 0.00682377
[115]	cv_agg's res: 0.791537 + 0.0048218
[120]	cv_agg's res: 0.791625 + 0.00584261
[125]	cv_agg's res: 0.792205 + 0.00454285
[130]	cv_agg's res: 0.789948 + 0.00546634
[135]	cv_agg's res: 0.790665 + 0.00507738
[140]	cv_agg's res: 0.788891 + 0.00629213
[145]	cv_agg's res: 0.788626 + 0.00572471
[150]	cv_agg's res: 0.787413 + 0.00466281
[155]	cv_agg's res: 0.788295 + 0.00619076
[160]	cv_agg's res: 0.787786 + 0.00618623
[165]

...

[110]	cv_agg's res: 0.789568 + 0.0042206
[115]	cv_agg's res: 0.789879 + 0.00353894
[120]	cv_agg's res: 0.790321 + 0.00290695
[125]	cv_agg's res: 0.787604 + 0.00248158
[130]	cv_agg's res: 0.785137 + 0.00217649
[135]	cv_agg's res: 0.785657 + 0.00196645
[140]	cv_agg's res: 0.784851 + 0.00507499
[145]	cv_agg's res: 0.785906 + 0.00331681
[150]	cv_agg's res: 0.783591 + 0.00432979
[155]	cv_agg's res: 0.784935 + 0.00568447
[160]	cv_agg's res: 0.784583 + 0.00590682
[165]	cv_agg's res: 0.784195 + 0.00538147
[170]	cv_agg's res: 0.783693 + 0.00647129
[175]	cv_agg's res: 0.784649 + 0.00589651
[180]	cv_agg's res: 0.783795 + 0.00515017
[185]	cv_agg's res: 0.784023 + 0.00618268
[190]	cv_agg's res: 0.785389 + 0.00623189
[195]	cv_agg's res: 0.786428 + 0.00571853
[200]	cv_agg's res: 0.787324 + 0.00412716
[205]	cv_agg's res: 0.787212 + 0.00505513
[210]	cv_agg's res: 0.786536 + 0.00579657
0.7914223067149889
[5]	cv_agg's res: 0.688345 + 0.0169154
[10]	cv_agg's res: 0.701109 + 0.0091709
[15]	cv_agg's res: 0.

[385]	cv_agg's res: 0.76333 + 0.0062623
[390]	cv_agg's res: 0.763163 + 0.00502111
[395]	cv_agg's res: 0.76205 + 0.00465154
[400]	cv_agg's res: 0.760743 + 0.00455701
[405]	cv_agg's res: 0.760441 + 0.0060386
[410]	cv_agg's res: 0.7588 + 0.00499581
0.7646271074862051
[5]	cv_agg's res: 0.705364 + 0.0121201
[10]	cv_agg's res: 0.736673 + 0.00902021
[15]	cv_agg's res: 0.753727 + 0.00191317
[20]	cv_agg's res: 0.767721 + 0.00205713
[25]	cv_agg's res: 0.776883 + 0.00204218
[30]	cv_agg's res: 0.781807 + 0.0033675
[35]	cv_agg's res: 0.784972 + 0.00245724
[40]	cv_agg's res: 0.790281 + 0.00183876
[45]	cv_agg's res: 0.788523 + 0.00368567
[50]	cv_agg's res: 0.790814 + 0.00333403
[55]	cv_agg's res: 0.789682 + 0.0043669
[60]	cv_agg's res: 0.790393 + 0.00525618
[65]	cv_agg's res: 0.789939 + 0.00495608
[70]	cv_agg's res: 0.788892 + 0.00596651
[75]	cv_agg's res: 0.789839 + 0.00637734
[80]	cv_agg's res: 0.789336 + 0.00550621
[85]	cv_agg's res: 0.790245 + 0.00433869
[90]	cv_agg's res: 0.789701 + 0.00358067
[

[105]	cv_agg's res: 0.788615 + 0.00366211
[110]	cv_agg's res: 0.789568 + 0.0042206
[115]	cv_agg's res: 0.789879 + 0.00353894
[120]	cv_agg's res: 0.790321 + 0.00290695
[125]	cv_agg's res: 0.787604 + 0.00248158
[130]	cv_agg's res: 0.785137 + 0.00217649
[135]	cv_agg's res: 0.785657 + 0.00196645
[140]	cv_agg's res: 0.784851 + 0.00507499
[145]	cv_agg's res: 0.785906 + 0.00331681
[150]	cv_agg's res: 0.783591 + 0.00432979
[155]	cv_agg's res: 0.784935 + 0.00568447
[160]	cv_agg's res: 0.784583 + 0.00590682
[165]	cv_agg's res: 0.784195 + 0.00538147
[170]	cv_agg's res: 0.783693 + 0.00647129
[175]	cv_agg's res: 0.784649 + 0.00589651
[180]	cv_agg's res: 0.783795 + 0.00515017
[185]	cv_agg's res: 0.784023 + 0.00618268
[190]	cv_agg's res: 0.785389 + 0.00623189
[195]	cv_agg's res: 0.786428 + 0.00571853
[200]	cv_agg's res: 0.787324 + 0.00412716
[205]	cv_agg's res: 0.787212 + 0.00505513
[210]	cv_agg's res: 0.786536 + 0.00579657
0.7914223067149889
[5]	cv_agg's res: 0.714849 + 0.0103969
[10]	cv_agg's res: 

[390]	cv_agg's res: 0.770736 + 0.00190232
0.7772075758282414
[5]	cv_agg's res: 0.705364 + 0.0121201
[10]	cv_agg's res: 0.736673 + 0.00902021
[15]	cv_agg's res: 0.753727 + 0.00191317
[20]	cv_agg's res: 0.767721 + 0.00205713
[25]	cv_agg's res: 0.776883 + 0.00204218
[30]	cv_agg's res: 0.781807 + 0.0033675
[35]	cv_agg's res: 0.784972 + 0.00245724
[40]	cv_agg's res: 0.790281 + 0.00183876
[45]	cv_agg's res: 0.788523 + 0.00368567
[50]	cv_agg's res: 0.790814 + 0.00333403
[55]	cv_agg's res: 0.789682 + 0.0043669
[60]	cv_agg's res: 0.790393 + 0.00525618
[65]	cv_agg's res: 0.789939 + 0.00495608
[70]	cv_agg's res: 0.788892 + 0.00596651
[75]	cv_agg's res: 0.789839 + 0.00637734
[80]	cv_agg's res: 0.789336 + 0.00550621
[85]	cv_agg's res: 0.790245 + 0.00433869
[90]	cv_agg's res: 0.789701 + 0.00358067
[95]	cv_agg's res: 0.788897 + 0.00203975
[100]	cv_agg's res: 0.790241 + 0.00224881
[105]	cv_agg's res: 0.788615 + 0.00366211
[110]	cv_agg's res: 0.789568 + 0.0042206
[115]	cv_agg's res: 0.789879 + 0.003538

[135]	cv_agg's res: 0.789109 + 0.0033064
[140]	cv_agg's res: 0.790676 + 0.00179818
[145]	cv_agg's res: 0.789361 + 0.00463539
[150]	cv_agg's res: 0.789644 + 0.00415911
[155]	cv_agg's res: 0.787583 + 0.00293326
[160]	cv_agg's res: 0.787728 + 0.00321044
[165]	cv_agg's res: 0.789314 + 0.00329471
[170]	cv_agg's res: 0.786827 + 0.00397762
[175]	cv_agg's res: 0.788015 + 0.00405505
[180]	cv_agg's res: 0.787622 + 0.00184457
[185]	cv_agg's res: 0.788624 + 0.00233957
[190]	cv_agg's res: 0.786177 + 0.00267089
[195]	cv_agg's res: 0.785884 + 0.00244711
[200]	cv_agg's res: 0.786756 + 0.00220531
[205]	cv_agg's res: 0.787164 + 0.00189695
[210]	cv_agg's res: 0.786133 + 0.00238966
[215]	cv_agg's res: 0.786758 + 0.000843689
[220]	cv_agg's res: 0.786507 + 0.0027095
[225]	cv_agg's res: 0.784497 + 0.00210297
[230]	cv_agg's res: 0.785682 + 0.00134171
[235]	cv_agg's res: 0.783516 + 0.000916564
0.791894063979571
[5]	cv_agg's res: 0.70768 + 0.0124215
[10]	cv_agg's res: 0.737108 + 0.0141958
[15]	cv_agg's res: 0.7

In [158]:
print(params['feature_fraction'], params['bagging_fraction'], params['bagging_freq'])

(0.6, 0.5, 0)


# 调参4

In [160]:
for lambda_l1 in [0.0,0.1,0.2,0.3,0.4]:
    for lambda_l2 in [0.0,0.1,0.2,0.3,0.4]:
        for min_split_gain in [0.0,0.1,0.2,0.3,0.4]:
            params['lambda_l1'] = lambda_l1
            params['lambda_l2'] = lambda_l2
            params['min_split_gain'] = min_split_gain
            
            cv_results = lgb.cv(
                            params,
                            lgb_train,
                            seed=42,
                            nfold=3,
                            feval=evalMetric,
                            metrics=['evalMetric'],
                            early_stopping_rounds=100,
                            verbose_eval=5,
                            num_boost_round=10000,
                            )
                    
            res_mean = pd.Series(cv_results['res-mean']).max()
            # boost_rounds = pd.Series(cv_results['binary_error-mean']).argmin()
            print(res_mean)

            if res_mean > max_mean:
                print(lambda_l1, lambda_l2, min_split_gain)
                max_mean = res_mean
                best_params['lambda_l1'] = lambda_l1
                best_params['lambda_l2'] = lambda_l2
                best_params['min_split_gain'] = min_split_gain

params['lambda_l1'] = best_params['lambda_l1']
params['lambda_l2'] = best_params['lambda_l2']
params['min_split_gain'] = best_params['min_split_gain']

[5]	cv_agg's res: 0.708098 + 0.00844165
[10]	cv_agg's res: 0.744272 + 0.00537953
[15]	cv_agg's res: 0.761955 + 0.00402213
[20]	cv_agg's res: 0.770229 + 0.00500564
[25]	cv_agg's res: 0.774722 + 0.00160573
[30]	cv_agg's res: 0.779738 + 0.00387914
[35]	cv_agg's res: 0.781562 + 0.00366879
[40]	cv_agg's res: 0.78411 + 0.00295836
[45]	cv_agg's res: 0.78635 + 0.00333523
[50]	cv_agg's res: 0.788963 + 0.00311815
[55]	cv_agg's res: 0.789113 + 0.00365299
[60]	cv_agg's res: 0.791287 + 0.00251886
[65]	cv_agg's res: 0.792441 + 0.00144544
[70]	cv_agg's res: 0.792773 + 0.000842747
[75]	cv_agg's res: 0.793294 + 0.00113167
[80]	cv_agg's res: 0.793012 + 0.00146955
[85]	cv_agg's res: 0.791605 + 0.000874739
[90]	cv_agg's res: 0.792326 + 0.00113989
[95]	cv_agg's res: 0.79231 + 0.00243053
[100]	cv_agg's res: 0.792195 + 0.00418949
[105]	cv_agg's res: 0.793481 + 0.00525199
[110]	cv_agg's res: 0.792936 + 0.00470461
[115]	cv_agg's res: 0.791323 + 0.00428579
[120]	cv_agg's res: 0.792044 + 0.00418448
[125]	cv_agg'

[140]	cv_agg's res: 0.791059 + 0.00409818
[145]	cv_agg's res: 0.791178 + 0.00432055
[150]	cv_agg's res: 0.79007 + 0.00301919
[155]	cv_agg's res: 0.789635 + 0.00414433
[160]	cv_agg's res: 0.790258 + 0.00421313
[165]	cv_agg's res: 0.79093 + 0.00524382
[170]	cv_agg's res: 0.790374 + 0.00550196
0.7954947611444588
[5]	cv_agg's res: 0.708098 + 0.00844165
[10]	cv_agg's res: 0.744407 + 0.00524498
[15]	cv_agg's res: 0.761953 + 0.00402572
[20]	cv_agg's res: 0.770223 + 0.00500247
[25]	cv_agg's res: 0.774708 + 0.00160685
[30]	cv_agg's res: 0.77974 + 0.00387102
[35]	cv_agg's res: 0.781556 + 0.00366669
[40]	cv_agg's res: 0.784106 + 0.00295767
[45]	cv_agg's res: 0.786346 + 0.00333428
[50]	cv_agg's res: 0.788962 + 0.00312047
[55]	cv_agg's res: 0.789112 + 0.00365398
[60]	cv_agg's res: 0.791285 + 0.0025232
[65]	cv_agg's res: 0.792434 + 0.00144088
[70]	cv_agg's res: 0.792774 + 0.000844222
[75]	cv_agg's res: 0.793305 + 0.00109898
[80]	cv_agg's res: 0.794665 + 0.000922085
[85]	cv_agg's res: 0.792382 + 0.00

[105]	cv_agg's res: 0.788729 + 0.0040226
[110]	cv_agg's res: 0.788149 + 0.00459136
[115]	cv_agg's res: 0.788005 + 0.0037894
[120]	cv_agg's res: 0.78975 + 0.00374559
[125]	cv_agg's res: 0.789207 + 0.00203735
[130]	cv_agg's res: 0.789695 + 0.00499137
[135]	cv_agg's res: 0.78865 + 0.0027003
[140]	cv_agg's res: 0.788566 + 0.00298913
[145]	cv_agg's res: 0.789719 + 0.00344915
[150]	cv_agg's res: 0.787502 + 0.00268164
[155]	cv_agg's res: 0.787757 + 0.00115524
[160]	cv_agg's res: 0.788806 + 0.0024808
[165]	cv_agg's res: 0.786843 + 0.0020098
0.791794274000028
[5]	cv_agg's res: 0.708098 + 0.00844165
[10]	cv_agg's res: 0.74438 + 0.00526865
[15]	cv_agg's res: 0.762106 + 0.00381778
[20]	cv_agg's res: 0.770226 + 0.00498281
[25]	cv_agg's res: 0.774306 + 0.00123439
[30]	cv_agg's res: 0.780501 + 0.0040419
[35]	cv_agg's res: 0.780738 + 0.00309342
[40]	cv_agg's res: 0.783489 + 0.00229451
[45]	cv_agg's res: 0.785804 + 0.00226502
[50]	cv_agg's res: 0.788837 + 0.00218223
[55]	cv_agg's res: 0.789985 + 0.0007

[50]	cv_agg's res: 0.787755 + 0.00172925
[55]	cv_agg's res: 0.78933 + 0.00155948
[60]	cv_agg's res: 0.790048 + 0.00224211
[65]	cv_agg's res: 0.791065 + 0.00317125
[70]	cv_agg's res: 0.791051 + 0.000197841
[75]	cv_agg's res: 0.790379 + 0.00108251
[80]	cv_agg's res: 0.790756 + 0.00234472
[85]	cv_agg's res: 0.792046 + 0.00280621
[90]	cv_agg's res: 0.791442 + 0.0032413
[95]	cv_agg's res: 0.790831 + 0.00381999
[100]	cv_agg's res: 0.789601 + 0.00275841
[105]	cv_agg's res: 0.790695 + 0.00127596
[110]	cv_agg's res: 0.790685 + 0.00157752
[115]	cv_agg's res: 0.791373 + 0.00234526
[120]	cv_agg's res: 0.791653 + 0.0012959
[125]	cv_agg's res: 0.792256 + 0.0019498
[130]	cv_agg's res: 0.791059 + 0.00234992
[135]	cv_agg's res: 0.789859 + 0.0021488
[140]	cv_agg's res: 0.79046 + 0.0031172
[145]	cv_agg's res: 0.791023 + 0.00406422
[150]	cv_agg's res: 0.789833 + 0.00414179
[155]	cv_agg's res: 0.790115 + 0.00401423
[160]	cv_agg's res: 0.790074 + 0.00393067
[165]	cv_agg's res: 0.789071 + 0.00351668
[170]	cv

[170]	cv_agg's res: 0.788311 + 0.00390991
[175]	cv_agg's res: 0.788889 + 0.00448041
[180]	cv_agg's res: 0.788591 + 0.00488253
[185]	cv_agg's res: 0.787397 + 0.00422607
[190]	cv_agg's res: 0.786705 + 0.00498521
[195]	cv_agg's res: 0.78701 + 0.00297525
[200]	cv_agg's res: 0.787737 + 0.0030181
0.792036800629936
[5]	cv_agg's res: 0.708098 + 0.00844165
[10]	cv_agg's res: 0.744441 + 0.00525067
[15]	cv_agg's res: 0.762124 + 0.0038535
[20]	cv_agg's res: 0.770238 + 0.00502282
[25]	cv_agg's res: 0.771205 + 0.00329973
[30]	cv_agg's res: 0.778163 + 0.0045401
[35]	cv_agg's res: 0.777919 + 0.00325558
[40]	cv_agg's res: 0.783559 + 0.00155886
[45]	cv_agg's res: 0.785041 + 0.00281986
[50]	cv_agg's res: 0.786951 + 0.00271161
[55]	cv_agg's res: 0.788092 + 0.00324381
[60]	cv_agg's res: 0.788219 + 0.00365721
[65]	cv_agg's res: 0.790896 + 0.00337476
[70]	cv_agg's res: 0.790104 + 0.00139531
[75]	cv_agg's res: 0.789464 + 0.000589888
[80]	cv_agg's res: 0.789766 + 1.45526e-05
[85]	cv_agg's res: 0.789308 + 0.002

...

[65]	cv_agg's res: 0.789445 + 0.00284651
[70]	cv_agg's res: 0.788333 + 0.00269091
[75]	cv_agg's res: 0.788059 + 0.000916431
[80]	cv_agg's res: 0.789582 + 0.000569607
[85]	cv_agg's res: 0.789921 + 0.000614522
[90]	cv_agg's res: 0.790082 + 0.00110076
[95]	cv_agg's res: 0.790815 + 0.00254488
[100]	cv_agg's res: 0.78925 + 0.00289458
[105]	cv_agg's res: 0.78752 + 0.00130704
[110]	cv_agg's res: 0.788694 + 0.00122759
[115]	cv_agg's res: 0.788358 + 0.000669596
[120]	cv_agg's res: 0.788796 + 0.00323021
[125]	cv_agg's res: 0.788939 + 0.00129785
[130]	cv_agg's res: 0.787413 + 0.00212296
[135]	cv_agg's res: 0.788577 + 0.00260184
[140]	cv_agg's res: 0.788617 + 0.00191093
[145]	cv_agg's res: 0.788291 + 0.00240415
[150]	cv_agg's res: 0.789203 + 0.00173418
[155]	cv_agg's res: 0.788184 + 0.000600789
[160]	cv_agg's res: 0.78812 + 0.000287345
[165]	cv_agg's res: 0.787025 + 0.00165146
[170]	cv_agg's res: 0.787361 + 0.0012251
[175]	cv_agg's res: 0.787114 + 0.00209568
[180]	cv_agg's res: 0.787218 + 0.002260

[45]	cv_agg's res: 0.784032 + 0.00282419
[50]	cv_agg's res: 0.786318 + 0.00324202
[55]	cv_agg's res: 0.787478 + 0.00289958
[60]	cv_agg's res: 0.789209 + 0.00416877
[65]	cv_agg's res: 0.790602 + 0.00262954
[70]	cv_agg's res: 0.790825 + 0.00213128
[75]	cv_agg's res: 0.791598 + 0.00234797
[80]	cv_agg's res: 0.792196 + 0.00309516
[85]	cv_agg's res: 0.792008 + 0.00325706
[90]	cv_agg's res: 0.790989 + 0.000696431
[95]	cv_agg's res: 0.791033 + 0.00024299
[100]	cv_agg's res: 0.791213 + 0.00141733
[105]	cv_agg's res: 0.792309 + 0.0019936
[110]	cv_agg's res: 0.790834 + 0.00269985
[115]	cv_agg's res: 0.7914 + 0.0032305
[120]	cv_agg's res: 0.790857 + 0.00299759
[125]	cv_agg's res: 0.791059 + 0.00324832
[130]	cv_agg's res: 0.789228 + 0.00485099
[135]	cv_agg's res: 0.790415 + 0.00297406
[140]	cv_agg's res: 0.7896 + 0.00223234
[145]	cv_agg's res: 0.789463 + 0.0026707
[150]	cv_agg's res: 0.789989 + 0.002725
[155]	cv_agg's res: 0.790201 + 0.00349292
[160]	cv_agg's res: 0.78943 + 0.00325553
[165]	cv_agg

[85]	cv_agg's res: 0.791545 + 0.00120231
[90]	cv_agg's res: 0.789687 + 0.000245851
[95]	cv_agg's res: 0.790943 + 0.00165123
[100]	cv_agg's res: 0.790243 + 0.00189044
[105]	cv_agg's res: 0.790982 + 0.00342264
[110]	cv_agg's res: 0.790534 + 0.00259028
[115]	cv_agg's res: 0.790079 + 0.002519
[120]	cv_agg's res: 0.790096 + 0.00252973
[125]	cv_agg's res: 0.791096 + 0.00188887
[130]	cv_agg's res: 0.790482 + 0.000611673
[135]	cv_agg's res: 0.787847 + 0.00132005
[140]	cv_agg's res: 0.789226 + 0.000405319
[145]	cv_agg's res: 0.78861 + 0.00130327
[150]	cv_agg's res: 0.788941 + 0.00195427
[155]	cv_agg's res: 0.788662 + 0.00111961
[160]	cv_agg's res: 0.789259 + 0.00143354
[165]	cv_agg's res: 0.788734 + 0.00235341
[170]	cv_agg's res: 0.788372 + 0.00300344
[175]	cv_agg's res: 0.789109 + 0.00293663
[180]	cv_agg's res: 0.788948 + 0.00465649
[185]	cv_agg's res: 0.788256 + 0.00527378
0.793142063418121
[5]	cv_agg's res: 0.709042 + 0.00770129
[10]	cv_agg's res: 0.744443 + 0.00524563
[15]	cv_agg's res: 0.7

[110]	cv_agg's res: 0.787768 + 0.0019268
[115]	cv_agg's res: 0.787397 + 0.00196979
[120]	cv_agg's res: 0.788396 + 0.00252569
[125]	cv_agg's res: 0.788763 + 0.0022056
[130]	cv_agg's res: 0.787905 + 0.00188638
[135]	cv_agg's res: 0.788705 + 0.00296632
[140]	cv_agg's res: 0.788097 + 0.00301943
[145]	cv_agg's res: 0.787872 + 0.00341515
[150]	cv_agg's res: 0.788822 + 0.00358779
[155]	cv_agg's res: 0.787709 + 0.00494354
[160]	cv_agg's res: 0.788034 + 0.00383798
[165]	cv_agg's res: 0.789374 + 0.00452545
[170]	cv_agg's res: 0.789035 + 0.00433342
[175]	cv_agg's res: 0.788774 + 0.00456258
[180]	cv_agg's res: 0.78782 + 0.00415212
[185]	cv_agg's res: 0.787538 + 0.00392261
[190]	cv_agg's res: 0.787323 + 0.00391145
[195]	cv_agg's res: 0.788339 + 0.00328496
0.7910995374584017
[5]	cv_agg's res: 0.709042 + 0.00770129
[10]	cv_agg's res: 0.744442 + 0.00524684
[15]	cv_agg's res: 0.761967 + 0.00404689
[20]	cv_agg's res: 0.769509 + 0.00483637
[25]	cv_agg's res: 0.771297 + 0.00427699
[30]	cv_agg's res: 0.778

[165]	cv_agg's res: 0.788139 + 0.00419144
[170]	cv_agg's res: 0.789294 + 0.00518931
[175]	cv_agg's res: 0.789268 + 0.00481662
[180]	cv_agg's res: 0.788445 + 0.00447779
[185]	cv_agg's res: 0.788931 + 0.00444443
0.7905777310738819
[5]	cv_agg's res: 0.709042 + 0.00770129
[10]	cv_agg's res: 0.744441 + 0.00524804
[15]	cv_agg's res: 0.761971 + 0.00404492
[20]	cv_agg's res: 0.7695 + 0.00484116
[25]	cv_agg's res: 0.771369 + 0.00419901
[30]	cv_agg's res: 0.778382 + 0.00345616
[35]	cv_agg's res: 0.778075 + 0.0037623
[40]	cv_agg's res: 0.782963 + 0.00293904
[45]	cv_agg's res: 0.783623 + 0.0022254
[50]	cv_agg's res: 0.786206 + 0.00342038
[55]	cv_agg's res: 0.789093 + 0.0032634
[60]	cv_agg's res: 0.788655 + 0.00215864
[65]	cv_agg's res: 0.788581 + 0.0024338
[70]	cv_agg's res: 0.78816 + 0.00192131
[75]	cv_agg's res: 0.787567 + 0.00131078
[80]	cv_agg's res: 0.789013 + 0.00316706
[85]	cv_agg's res: 0.788326 + 0.00173475
[90]	cv_agg's res: 0.788413 + 0.00297285
[95]	cv_agg's res: 0.786942 + 0.00333737


In [161]:
print(best_params)

{'num_leaves': 50, 'lambda_l1': 0.0, 'bagging_freq': 0, 'learning_rate': 0.15000000000000002, 'lambda_l2': 0.1, 'min_split_gain': 0.2, 'min_data_in_leaf': 360, 'max_bin': 255, 'bagging_fraction': 0.5, 'feature_fraction': 0.6}


In [398]:
params['num_leaves'] = 50
params['lambda_l1'] = 0.0
params['bagging_freq'] = 0
params['learning_rate'] = 0.2
params['lambda_l2'] = 0.1
params['min_split_gain'] = 0.2
params['min_data_in_leaf'] = 360
params['max_bin'] = 255
params['bagging_fraction'] = 0.5
params['feature_fraction'] = 0.6

# Train

In [399]:
model = lgb.train(
          params,                     # 参数字典
          lgb_train,                  # 训练集
          valid_sets=lgb_eval,        # 验证集
          num_boost_round=2000,       # 迭代次数
          early_stopping_rounds=50,
        feval=evalMetric,
        verbose_eval=5
    )

Training until validation scores don't improve for 50 rounds.
[5]	valid_0's auc: 0.864601	valid_0's res: 0.744929
[10]	valid_0's auc: 0.883617	valid_0's res: 0.777259
[15]	valid_0's auc: 0.893798	valid_0's res: 0.784196
[20]	valid_0's auc: 0.9011	valid_0's res: 0.789081
[25]	valid_0's auc: 0.902832	valid_0's res: 0.789828
[30]	valid_0's auc: 0.905807	valid_0's res: 0.796567
[35]	valid_0's auc: 0.905596	valid_0's res: 0.799272
[40]	valid_0's auc: 0.907444	valid_0's res: 0.80038
[45]	valid_0's auc: 0.908723	valid_0's res: 0.802531
[50]	valid_0's auc: 0.910229	valid_0's res: 0.801964
[55]	valid_0's auc: 0.909524	valid_0's res: 0.802236
[60]	valid_0's auc: 0.90845	valid_0's res: 0.802921
[65]	valid_0's auc: 0.909187	valid_0's res: 0.807771
[70]	valid_0's auc: 0.910218	valid_0's res: 0.812059
[75]	valid_0's auc: 0.910345	valid_0's res: 0.812129
[80]	valid_0's auc: 0.909302	valid_0's res: 0.809293
[85]	valid_0's auc: 0.909871	valid_0's res: 0.810367
[90]	valid_0's auc: 0.909439	valid_0's res

# 线下预测

In [400]:
preds_offline = model.predict(offline_test_X, num_iteration=model.best_iteration)
offline=offline_test[['uid','label']]
offline.loc[:, 'preds']=preds_offline
offline.loc[:, 'label'] = offline['label'].astype(np.float64)
print('log_loss', metrics.log_loss(offline.label, offline.preds))

('log_loss', 0.35383034824181936)


# 线上预测

In [381]:
preds_online =  model.predict(online_test_X, num_iteration=model.best_iteration)

In [382]:
id_series = pd.DataFrame({'uid': test_a_df.uid})
def do3(x):
    tmp = "%04d" % x['uid']
    return 'u' + str(tmp)
id_series.loc[:, 'uid'] = id_series.apply(do3, axis = 1)

In [383]:
res =pd.DataFrame({'uid':id_series['uid'],'label':preds_online})

In [384]:
res=res.sort_values(by='label',ascending=False)
res.label=res.label.map(lambda x: 1 if x>=0.5 else 0)

In [385]:
print(res)

      label    uid
1145      1  u6145
378       1  u5378
1855      1  u6855
643       1  u5643
84        1  u5084
1157      1  u6157
607       1  u5607
1854      1  u6854
635       1  u5635
583       1  u5583
1099      1  u6099
555       1  u5555
603       1  u5603
1170      1  u6170
1405      1  u6405
149       1  u5149
1094      1  u6094
1404      1  u6404
1695      1  u6695
608       1  u5608
1820      1  u6820
1234      1  u6234
1119      1  u6119
658       1  u5658
715       1  u5715
636       1  u5636
1872      1  u6872
1961      1  u6961
588       1  u5588
1333      1  u6333
...     ...    ...
1530      0  u6530
944       0  u5944
1086      0  u6086
21        0  u5021
1766      0  u6766
482       0  u5482
477       0  u5477
1711      0  u6711
786       0  u5786
670       0  u5670
1947      0  u6947
947       0  u5947
26        0  u5026
1092      0  u6092
1385      0  u6385
745       0  u5745
1023      0  u6023
247       0  u5247
1567      0  u6567
1997      0  u6997
627       0 

In [386]:
res.to_csv('../result/lgb-baseline-selfparam-4.csv',index=False,header=False,sep=',',columns=['uid','label'])

# 保存模型

In [281]:
from sklearn.externals import joblib
joblib.dump(model,'model.pkl')

['model.pkl']

# Feature Importance

In [387]:
df = pd.DataFrame(X_train.columns.tolist(), columns=['feature'])
df['importance']=list(model.feature_importance())
df = df.sort_values(by='importance',ascending=False)
df.to_csv("../result/feature_score_3.csv",index=None,encoding='gbk')